# HDBSCAN should work well on embedding representations

In [1]:
import os
import re
import json
import hdbscan

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from collections import Counter
import matplotlib.pyplot as plt

import lib.helper as helper
import lib.embedding_models as reps

from importlib import reload

%matplotlib inline

## 1.  Retrieve Corpus

The corpus is being scraped by the "run_news_scrapes.py" script (and windows task scheduler) every 12 hours, a bit past midday and a bit past midnight.

The "bing" corpus are news titles and text extracts gotten from the bing news search API, using a few Home Office - related keywords.

The "RSS" corpus is plugged directly into a number of RSS feeds for world news sites and local british news sites, with no filters for news story types or subjects applied.

In [2]:
# Should be same path for all my PC's, it's where each scrape goes as a separate json file.
storage_path = "D:/Dropbox/news_crow/scrape_results"

# "bing" is targeted news search corpus, "RSS" is from specific world and local news feeds.
corpus_type = "disaster"

# Load up
corpus = helper.load_clean_corpus(storage_path, corpus_type)

# Make sure after cleaning etc it's indexed from 0
corpus.reset_index(inplace=True)
corpus.index.name = "node"

# See how it turned out
print(corpus.shape)
corpus.head()

Total files: 316
9.8 percent of files read.
19.6 percent of files read.
29.4 percent of files read.
39.2 percent of files read.
49.1 percent of files read.
58.9 percent of files read.
68.7 percent of files read.
78.5 percent of files read.
88.3 percent of files read.
98.1 percent of files read.
(22829, 9)


,index,title,summary,date,link,source_url,retrieval_timestamp,origin,clean_text
node,,,,,,,,,
0,0,West Midlands <b>flood</b> warnings prompt &#3...,Residents have been warned to &quot;remain vig...,2019-11-17T17:35:00.0000000Z,https://www.bbc.co.uk/news/uk-england-50451817,www.bbc.co.uk,2019-11-17 19:50:58.278878,bing_news_api,West Midlands flood warnings prompt ;remain vi...
1,1,New <b>flood</b> warnings issued with more hom...,The Environment Agency has a number of <b>floo...,2019-11-17T18:35:00.0000000Z,https://www.hulldailymail.co.uk/news/hull-east...,www.hulldailymail.co.uk,2019-11-17 19:50:58.278928,bing_news_api,New flood warnings issued with more homes at r...
2,2,UK weather forecast – More than 100 <b>flood</...,<b>FLOOD</b>-ravaged villages in the UK have b...,2019-11-17T13:45:00.0000000Z,https://www.thesun.co.uk/news/10342583/uk-weat...,www.thesun.co.uk,2019-11-17 19:50:58.278953,bing_news_api,UK weather forecast – More than 100 flood aler...
3,5,UK <b>flood</b> warning map: <b>Flood</b> chao...,The Environment Agency has issued 57 <b>flood<...,2019-11-17T16:38:00.0000000Z,https://www.express.co.uk/news/weather/1205629...,www.express.co.uk,2019-11-17 19:50:58.279028,bing_news_api,UK flood warning map: Flood chaos to continue ...
4,6,UK weather forecast: <b>Flood</b> chaos contin...,Despite some areas enduring their &#39;wettest...,2019-11-17T18:32:00.0000000Z,https://www.mirror.co.uk/news/uk-news/uk-weath...,www.mirror.co.uk,2019-11-17 19:50:58.279047,bing_news_api,UK weather forecast: Flood chaos continues wit...


## 2.  Build Text Model (Representation, eg; word2vec, entities list...)

- Trying with the world corpus and with the bing corpus, neither worked with InferSent.  Suspect the problem lies in the PCA step, which may not be working well on this high-dimensional (vector length = 4096) form.
- Summed keywords works rather better with the world corpus.
- Summed keywords still fail the bing/home office corpus, giving me a cluster about "immigration" and a cluster for the American Supreme Court.

In [3]:
# Windows didn't play nicely with the vector datasets, Some obscure encoding problem (python in Conda
# kept trying to decode using cp1252 regardless of whatever other options I specified!)
# Solution; rewrite file and drop any characters the Windows encoder refuses to recognise.
# I shouldn't loose too much info.
#with open('./lib/InferSent/dataset/fastText/crawl-300d-2M.vec', "r", encoding="cp1252", errors="ignore") as infile:
#    with open('./lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec', "wb") as outfile:
#        for line in infile:
#            outfile.write(line.encode('cp1252'))

In [4]:
#infersent = reps.InferSentModel(list(corpus['clean_text']),
#                                list(corpus['clean_text']),
#                                W2V_PATH = './lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec')

#embeddings = infersent.get_embeddings()

In [5]:
#reload(reps)

In [6]:
# Whereas this worked first time!
#glove = reps.NounGloveWordModel(list(corpus['clean_text']), list(corpus['clean_text']))

#embeddings = glove.get_embeddings()

In [7]:
# Turn that into a DF for me
#embeddings_df = pd.DataFrame({"clean_text": list(embeddings.keys()),
#                              "embeddings": list(embeddings.values())})
#embeddings_df.shape

In [8]:
#embeddings_df.head()

## 2a.  Try a really simple averaged word vector model!

With a complex noun extraction function 'cause that part's slow so I multi-threaded it.

In [9]:
from gensim.models import Word2Vec
import spacy
nlp = spacy.load('en_core_web_sm')
from gensim.models.phrases import Phrases, Phraser

# Set vector size
vec_size = 100

In [10]:
def get_phrased_nouns(sentences):
    """ Use spacy to get all of the actual entities, conjoin bigram nouns. """

    # Get the lists of nouns
    noun_lists = []
    for doc in sentences:
        parsed = nlp(doc)
        noun_lists.append([token.lemma_ for token in parsed if token.pos_ == 'PROPN'])

    # Build the phrase model
    phrases = Phrases(noun_lists, min_count=5, threshold=0.5)

    # Get the set of phrases present in the model
    results = []
    for nouns in noun_lists:
        results.append(phrases[nouns])

    return results

# Get phrase-conjoined, lemmatized tokens
test = get_phrased_nouns(corpus['clean_text'])

# Detect and conjoin bigrams
model = Word2Vec(test, size=vec_size, window=5, min_count=1, workers=10)

In [11]:
def get_averaged_vec(token_list, model):
    
    vecs = []
    for token in token_list:
        try:
            vector = model.wv[token]
        except: 
            vector = np.zeros(vec_size)
        vecs.append(vector)
    
    if len(vecs) > 0:
        return np.mean(np.asarray(vecs), axis=0)
    else:
        return np.zeros(vec_size)

In [12]:
vectors = [get_averaged_vec(tokens, model) for tokens in test]

In [13]:
vectors[0:2]

[array([ 1.48773998e-01,  2.19226275e-02, -3.69016230e-02, -4.59784502e-03,
        -1.72700986e-01, -1.65858969e-01, -3.11440527e-02, -1.42761201e-01,
         1.06054032e-02, -1.68504547e-02, -1.70641556e-01, -1.19400948e-01,
         4.70643789e-02, -2.46798224e-03,  4.18019434e-03,  8.75702314e-03,
        -3.80119425e-03, -5.69423586e-02,  6.81633204e-02,  5.44520095e-02,
         2.96682157e-02, -1.36330946e-05, -8.81288722e-02, -1.33267462e-01,
        -2.70807743e-02,  3.99522074e-02, -7.35704973e-02,  2.28855133e-01,
         1.59268320e-01,  3.72315012e-02, -1.56046022e-02,  6.45692125e-02,
        -1.74158141e-02,  1.06112570e-01, -1.07601017e-01,  4.16241884e-02,
         1.50490955e-01, -3.47237550e-02,  2.55287569e-02,  3.37276384e-02,
        -1.30607402e-02, -8.68300498e-02, -6.76510409e-02, -3.97584364e-02,
        -1.71207830e-01, -6.84899017e-02,  1.20375462e-01,  3.05474699e-02,
        -1.35735288e-01, -1.27561735e-02,  2.92679705e-02,  2.52623349e-01,
        -2.2

## 3. Cluster Text

This is the part where the pipelines get a little more experimental
- First; PCA (HDBSCAN prefers < 50 dimensions if possible) (also, try just fitting 50 vectors!)
- Second; HDBSCAN clustering

In [19]:
embeddings_array = np.vstack(vectors)

# First, PCA the data
pca = PCA(n_components=20, svd_solver='full')

# Fit and check
embeddings_pca = pca.fit_transform(embeddings_array)

# Diagnostic stats - both should show exponential decay
print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

[9.98942139e-01 6.47943541e-04 1.61067927e-04 5.43628795e-05
 2.95846730e-05 1.52634150e-05 1.06385855e-05 9.79740789e-06
 7.34321907e-06 5.90516687e-06 4.72472700e-06 4.29751943e-06
 4.11768634e-06 3.62851272e-06 3.47705860e-06 3.27803462e-06
 2.99175525e-06 2.89244131e-06 2.70770930e-06 2.34930155e-06]
[227.67326277   5.79843038   2.89098873   1.67955059   1.23901125
   0.88995433   0.74299144   0.71301302   0.61728412   0.5535514
   0.49514252   0.47222697   0.46224103   0.43391652   0.42476416
   0.41242847   0.39400787   0.38741296   0.37483737   0.34914936]


In [20]:
# Clusterer fitting
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=10)
clusterer.fit(embeddings_array)

# Examine results
Counter(clusterer.labels_)

Counter({68: 10690,
         -1: 10399,
         2: 40,
         29: 5,
         35: 5,
         15: 12,
         51: 9,
         43: 7,
         54: 323,
         34: 5,
         12: 11,
         13: 33,
         28: 9,
         11: 22,
         49: 11,
         16: 10,
         18: 5,
         1: 6,
         41: 16,
         33: 7,
         37: 5,
         22: 13,
         24: 6,
         19: 13,
         3: 12,
         23: 9,
         42: 6,
         64: 53,
         0: 103,
         60: 6,
         32: 108,
         57: 10,
         26: 35,
         27: 9,
         17: 29,
         46: 27,
         69: 176,
         7: 10,
         5: 41,
         14: 15,
         48: 27,
         53: 34,
         10: 25,
         58: 12,
         67: 26,
         20: 10,
         50: 23,
         31: 5,
         65: 37,
         63: 22,
         62: 8,
         59: 14,
         66: 24,
         30: 19,
         47: 10,
         25: 5,
         9: 36,
         45: 15,
         4: 30,
         40: 

In [21]:
# Diagnostic - calculate percentage of records assigned "outlier"
100.0 * sum(clusterer.labels_ == -1) / clusterer.labels_.shape[0]

45.551710543606816

In [22]:
len(pd.unique(clusterer.labels_))

71

In [23]:
# Record cluster each was assigned to
corpus['cluster'] = clusterer.labels_

# Record the reverse of outlier score (therefore, higher number = higher certainty of membership)
corpus['score'] = 1.0 - clusterer.outlier_scores_

corpus.to_csv("working/disaster_clustered_w2v_direct.csv", index=False)

C:\ProgramData\Anaconda3\lib\site-packages\hdbscan\hdbscan_.py:1052: RuntimeWarning: invalid value encountered in double_scalars
  self._outlier_scores = outlier_scores(self._condensed_tree)
